In [22]:
import os
import subprocess
from config import Config
print(Config.SDE_CONNECTION)

from arcgis.gis import GIS
from arcgis.features import FeatureLayer

gis = GIS(Config.PORTAL_URL, Config.PORTAL_USER, Config.PORTAL_PASSWORD)


K:/webmaps/basemap/cc-gis.sde


In [23]:
def read_shape_data(gis):
    key_fields = ['DocumentNa']
    popup_fields = ["SYEAR", "SurveyDate", "BookPage", "Client", "Firm", "SurveyorKe"]

    # Using arcgis, I have to publish on Portal but then I can use spatial data.
    url = "https://delta.co.clatsop.or.us/server/rest/services/Public_Works/Surveys/FeatureServer/0"
    layer = FeatureLayer(url, gis)
    df = layer.query(where="1=1", out_fields=key_fields+popup_fields).sdf

    return df
survey_df = read_shape_data(gis)

In [36]:
import arcpy
import pandas as pd
images = os.path.join(Config.SDE_CONNECTION, "Clatsop.dbo.SURVEYIMAGES")
n = arcpy.da.TableToNumPyArray(in_table=images, field_names=["DocumentName", "Image", "DateModified"], skip_nulls=False)
docs_df = pd.DataFrame(n)
survey_df, docs_df


(       OBJECTID_1 DocumentNa SYEAR          SurveyDate BookPage  \
 0               1    BT-2039  2007 2007-01-31 08:00:00     None   
 1               2    BT-2040  2006 2006-08-10 07:00:00     None   
 2               3    BT-2041  2007 2007-02-21 08:00:00     None   
 3               4    BT-2042  2007 2007-05-17 07:00:00     None   
 4               5    BT-2043  2007 2007-05-21 07:00:00     None   
 ...           ...        ...   ...                 ...      ...   
 18711      115274   CS 13980  2021 2021-09-14 07:00:00     None   
 18712      115275   CS 13981  2021 2021-05-18 07:00:00     None   
 18713      115674    BT-2326  2021 2021-09-29 07:00:00     None   
 18714      115675   CS 00091  2021 1903-04-23 07:00:00     None   
 18715      115677   CS 13982  2021 2021-08-12 07:00:00     None   
 
                         Client                       Firm   SurveyorKe  \
 0               CLATSOP COUNTY      Clatsop Cnty Surveyor     Thornton   
 1               Clatsop County 

In [37]:
x = pd.merge(survey_df, docs_df, left_on="DocumentNa", right_on="DocumentName", validate="m:m")
x

,OBJECTID_1,DocumentNa,SYEAR,SurveyDate,BookPage,Client,Firm,SurveyorKe,SHAPE,DocumentName,Image,DateModified
0,1,BT-2039,2007,2007-01-31 08:00:00,None,CLATSOP COUNTY,Clatsop Cnty Surveyor,Thornton,"{'rings': [[[-13785453.635334412, 5806428.6626...",BT-2039,\\Clatsop.co.clatsop.or.us\data\Applications\S...,2007-02-12 07:41:40
1,2,BT-2040,2006,2006-08-10 07:00:00,None,Clatsop County,Clatsop Cnty Surveyor,Thornton,"{'rings': [[[-13754112.821037361, 5808043.9623...",BT-2040,\\Clatsop.co.clatsop.or.us\data\Applications\S...,2007-02-21 14:13:34
2,3,BT-2041,2007,2007-02-21 08:00:00,None,Clatsop County,Clatsop Cnty Surveyor,Thornton,"{'rings': [[[-13786602.409381496, 5803018.0689...",BT-2041,\\Clatsop.co.clatsop.or.us\data\Applications\S...,2007-03-07 09:01:36
3,4,BT-2042,2007,2007-05-17 07:00:00,None,,Castle Rock Surveying,Wickman,"{'rings': [[[-13769574.754235515, 5751040.1368...",BT-2042,\\Clatsop.co.clatsop.or.us\data\Applications\S...,2007-05-29 13:09:06
4,5,BT-2043,2007,2007-05-21 07:00:00,None,,Castle Rock Surveying,Wickman,"{'rings': [[[-13770637.77574605, 5752142.87608...",BT-2043,\\Clatsop.co.clatsop.or.us\data\Applications\S...,2007-05-29 13:09:38
...,...,...,...,...,...,...,...,...,...,...,...,...
23461,115275,CS 13981,2021,2021-05-18 07:00:00,None,Rose Family Trust,Mendenhall and Associates,Mendenhall,"{'rings': [[[-13795769.506285012, 5791766.3300...",CS 13981,\\clatsop.co.clatsop.or.us\data\Applications\S...,2021-11-02 08:26:14
23462,115275,CS 13981,2021,2021-05-18 07:00:00,None,Rose Family Trust,Mendenhall and Associates,Mendenhall,"{'rings': [[[-13795769.506285012, 5791766.3300...",CS 13981,\\clatsop.co.clatsop.or.us\data\Applications\S...,2021-11-02 08:24:24
23463,115674,BT-2326,2021,2021-09-29 07:00:00,None,Clatsop County Surveyor,Clatsop County Surveyor,Swenson,"{'rings': [[[-13794705.524035223, 5803005.1340...",BT-2326,\\clatsop.co.clatsop.or.us\data\Applications\S...,2021-11-05 10:11:06
23464,115675,CS 00091,2021,1903-04-23 07:00:00,None,Clatsop County Surveyor,Clatsop County Survryor,Astbury,"{'rings': [[[-13798833.320253417, 5812862.9486...",CS 00091,\\clatsop.co.clatsop.or.us\data\Applications\S...,2021-11-02 12:10:49


In [38]:
basepath = "\\\\clatsop.co.clatsop.or.us\\data\\applications\\surveyordata\\survey\\scanned surveys\\aa_indexed_surveys\\"
groups = x.groupby('DocumentNa')
surveys2 = {}
for name,group in groups:
    longimages = group["Image"].tolist()
    surveys2[name] = [os.path.splitext(item)[0].lower().replace(basepath,'').replace('\\', '/') for item in longimages]  
surveys2


{'Adair, John DLC Towns and Subs within': ['subdivisions_by_name_all/adair, john dlc towns and subs within b0 p53 copy',
  'subdivisions_by_name_all/adair, john dlc towns and subs within b0 p53',
  'subdivisions_by_name_all/adair, john dlc towns and subs within b0 p54 copy',
  'subdivisions_by_name_all/adair, john dlc towns and subs within b0 p54',
  'subdivisions_by_name_all/adair, john dlc towns and subs within b0 p55',
  'subdivisions_by_name_all/adairs astoria b02 p19 copy',
  'subdivisions_by_name_all/adairs astoria b02 p19',
  'subdivisions_by_name_all/adairs astoria b02 p20 copy',
  'subdivisions_by_name_all/adairs astoria b02 p20'],
 'Adairs Astoria, Amended Plat of Blocks 24,26,60 and 68': ['subdivisions_by_name_all/adairs astoria, amended plat of blocks 24,26,60 and 68 b0 p57 copy',
  'subdivisions_by_name_all/adairs astoria, amended plat of blocks 24,26,60 and 68 b0 p57'],
 'Adairs Astoria, Asmus Brixs Sub of Lots 3 and 4, Block 69': ['subdivisions_by_name_all/adairs astoria

In [27]:
surveys3 = list()
baseurl = "https://delta.co.clatsop.or.us/survey/"
maxlink=0
for k,images in surveys2.items():
    link = 0
    d = dict()
    d['DocumentNa'] = k
    for baselink in sorted(images):
        link += 1
        name = os.path.split(baselink)[1]
        d['SURVEY%02d' % link] = name
        d['HYPERLINK%02d' % link] = baseurl + baselink + '.pdf'
    maxlink = max(maxlink, link)
    d['HYPERLINKS'] = maxlink
    surveys3.append(d)
surveys3

[{'DocumentNa': 'Adair, John DLC Towns and Subs within',
  'SURVEY01': 'adair, john dlc towns and subs within b0 p53',
  'HYPERLINK01': 'https://delta.co.clatsop.or.us/survey/subdivisions_by_name_all/adair, john dlc towns and subs within b0 p53.pdf',
  'SURVEY02': 'adair, john dlc towns and subs within b0 p53 copy',
  'HYPERLINK02': 'https://delta.co.clatsop.or.us/survey/subdivisions_by_name_all/adair, john dlc towns and subs within b0 p53 copy.pdf',
  'SURVEY03': 'adair, john dlc towns and subs within b0 p54',
  'HYPERLINK03': 'https://delta.co.clatsop.or.us/survey/subdivisions_by_name_all/adair, john dlc towns and subs within b0 p54.pdf',
  'SURVEY04': 'adair, john dlc towns and subs within b0 p54 copy',
  'HYPERLINK04': 'https://delta.co.clatsop.or.us/survey/subdivisions_by_name_all/adair, john dlc towns and subs within b0 p54 copy.pdf',
  'SURVEY05': 'adair, john dlc towns and subs within b0 p55',
  'HYPERLINK05': 'https://delta.co.clatsop.or.us/survey/subdivisions_by_name_all/adai

In [28]:
hyperlinks_df=pd.DataFrame.from_dict(surveys3)

In [29]:
xl = pd.merge(survey_df, hyperlinks_df, on="DocumentNa")
xl

,OBJECTID_1,DocumentNa,SYEAR,SurveyDate,BookPage,Client,Firm,SurveyorKe,SHAPE,SURVEY01,...,HYPERLINK09,HYPERLINKS,SURVEY10,HYPERLINK10,SURVEY11,HYPERLINK11,SURVEY12,HYPERLINK12,SURVEY13,HYPERLINK13
0,1,BT-2039,2007,2007-01-31 08:00:00,None,CLATSOP COUNTY,Clatsop Cnty Surveyor,Thornton,"{'rings': [[[-13785453.635334412, 5806428.6626...",bt-2039,...,NaN,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,BT-2040,2006,2006-08-10 07:00:00,None,Clatsop County,Clatsop Cnty Surveyor,Thornton,"{'rings': [[[-13754112.821037361, 5808043.9623...",bt-2040,...,NaN,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,BT-2041,2007,2007-02-21 08:00:00,None,Clatsop County,Clatsop Cnty Surveyor,Thornton,"{'rings': [[[-13786602.409381496, 5803018.0689...",bt-2041,...,NaN,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,BT-2042,2007,2007-05-17 07:00:00,None,,Castle Rock Surveying,Wickman,"{'rings': [[[-13769574.754235515, 5751040.1368...",bt-2042,...,NaN,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,BT-2043,2007,2007-05-21 07:00:00,None,,Castle Rock Surveying,Wickman,"{'rings': [[[-13770637.77574605, 5752142.87608...",bt-2043,...,NaN,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18592,115274,CS 13980,2021,2021-09-14 07:00:00,None,Jeff Vahid-Tari,Benthin Land Surveying,Benthin,"{'rings': [[[-13794996.79819891, 5779700.73983...",cs 13980,...,NaN,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18593,115275,CS 13981,2021,2021-05-18 07:00:00,None,Rose Family Trust,Mendenhall and Associates,Mendenhall,"{'rings': [[[-13795769.506285012, 5791766.3300...",cs 13981 1-2,...,NaN,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18594,115674,BT-2326,2021,2021-09-29 07:00:00,None,Clatsop County Surveyor,Clatsop County Surveyor,Swenson,"{'rings': [[[-13794705.524035223, 5803005.1340...",bt-2326,...,NaN,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18595,115675,CS 00091,2021,1903-04-23 07:00:00,None,Clatsop County Surveyor,Clatsop County Survryor,Astbury,"{'rings': [[[-13798833.320253417, 5812862.9486...",cs 00091,...,NaN,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
from utils import get_files_df
import os
os.chdir(Config.SOURCE)
files_df = get_files_df('.')
files_df

,path,file,ext,pathname,lastmod,match
0,.,Thumbs,db,.\Thumbs.db,2019-08-27 09:54:42.689000,False
1,.,web,config,.\web.config,2019-10-14 09:38:02.965713,False
2,.\00000-00999,CS 00147,tif,.\00000-00999\CS 00147.tif,2007-05-08 09:44:46.000000,True
3,.\00000-00999,CS 00049,pdf,.\00000-00999\CS 00049.pdf,2014-09-30 08:25:19.399836,True
4,.\00000-00999,CS 00055,pdf,.\00000-00999\CS 00055.pdf,2016-12-01 08:34:15.668217,True
...,...,...,...,...,...,...
22907,.\Subdivisions_by_Name_All,Woodland Park Add to Gearhart Park Blks 1-6 B5...,jpg,.\Subdivisions_by_Name_All\Woodland Park Add t...,2018-07-09 15:55:11.772368,True
22908,.\Subdivisions_by_Name_All,Youngs River Cemetary B01 P63 copy,tif,.\Subdivisions_by_Name_All\Youngs River Cemeta...,2019-05-07 16:27:50.853728,True
22909,.\Subdivisions_by_Name_All,Youngs River Cemetary B01 P63,jpg,.\Subdivisions_by_Name_All\Youngs River Cemeta...,2019-01-09 14:11:11.597847,True
22910,.\Subdivisions_by_Name_All,~annon Beach Park B03 P59 copy,tmp,.\Subdivisions_by_Name_All\~annon Beach Park B...,2021-06-11 12:26:18.220292,False


In [31]:
def fixdocname(s):
    pathname = s.Image
    p,fe = os.path.split(pathname)
    f,e = os.path.splitext(fe)
    s['Doc'] = f
    return s
docs_short_df = docs_df.apply(fixdocname, axis=1)
docs_short_df

,DocumentName,Image,DateModified,Doc
0,BT-2179,\\Clatsop.co.clatsop.or.us\data\Applications\S...,2010-05-03 10:17:16.000000,BT-2179
1,BT-2180,\\Clatsop.co.clatsop.or.us\data\Applications\S...,2010-04-29 09:54:46.000000,BT-2180
2,BT-2181,\\Clatsop.co.clatsop.or.us\data\Applications\S...,2010-05-18 15:33:16.000000,BT-2181
3,BT-2182,\\Clatsop.co.clatsop.or.us\data\Applications\S...,2010-05-18 15:34:02.000000,BT-2182
4,BT-2183,\\Clatsop.co.clatsop.or.us\data\Applications\S...,2010-06-14 08:16:52.000000,BT-2183
...,...,...,...,...
24009,CS 13513,\\clatsop.co.clatsop.or.us\data\Applications\S...,2018-09-28 10:59:24.000000,CS 13513
24010,CS 13406,\\clatsop.co.clatsop.or.us\data\Applications\S...,2018-06-19 15:00:57.000000,CS 13406 7-7
24011,CS 13483,\\clatsop.co.clatsop.or.us\data\Applications\S...,2018-07-12 16:52:37.000000,CS 13483
24012,P2018-012,\\clatsop.co.clatsop.or.us\data\Applications\S...,2018-06-21 12:35:23.000001,P2018-012 1-2


In [32]:
xdocs = pd.merge(docs_short_df, files_df, left_on='Doc', right_on='file')
xdocs[xdocs['match']==False]

,DocumentName,Image,DateModified,Doc,path,file,ext,pathname,lastmod,match
9917,CS 13797,\\clatsop.co.clatsop.or.us\data\Applications\S...,2019-10-14 09:38:02.000000,web,.,web,config,.\web.config,2019-10-14 09:38:02.965713,False
10440,Unknown,\\clatsop.co.clatsop.or.us\data\Applications\S...,2021-11-02 11:17:29.000001,Alderbrook B0 P25 copy.tif.aux,.\Subdivisions_by_Name_All,Alderbrook B0 P25 copy.tif.aux,xml,.\Subdivisions_by_Name_All\Alderbrook B0 P25 c...,2021-11-02 11:17:29.028188,False
10441,Unknown,\\clatsop.co.clatsop.or.us\data\Applications\S...,2021-11-02 11:17:28.000000,Alderbrook B0 P25 copy.tif,.\Subdivisions_by_Name_All,Alderbrook B0 P25 copy.tif,ovr,.\Subdivisions_by_Name_All\Alderbrook B0 P25 c...,2021-11-02 11:17:28.043725,False
10442,Unknown,\\clatsop.co.clatsop.or.us\data\Applications\S...,2021-11-02 11:23:39.000000,Alderbrook B0 P26 copy.tif.aux,.\Subdivisions_by_Name_All,Alderbrook B0 P26 copy.tif.aux,xml,.\Subdivisions_by_Name_All\Alderbrook B0 P26 c...,2021-11-02 11:23:39.608002,False
10443,Unknown,\\clatsop.co.clatsop.or.us\data\Applications\S...,2021-11-02 11:23:35.000000,Alderbrook B0 P26 copy.tif,.\Subdivisions_by_Name_All,Alderbrook B0 P26 copy.tif,ovr,.\Subdivisions_by_Name_All\Alderbrook B0 P26 c...,2021-11-02 11:23:35.774943,False
11855,Williamsport,\\clatsop.co.clatsop.or.us\data\Applications\S...,2019-01-08 15:19:48.000000,Williamsport B0 P23,.\Subdivisions_by_Name_All,Williamsport B0 P23,jgwx,.\Subdivisions_by_Name_All\Williamsport B0 P23...,2021-11-09 14:48:03.499516,False
12293,UNKNOWN,\\clatsop.co.clatsop.or.us\data\Applications\S...,2001-05-25 08:34:34.000000,BT-1655 TO BT-2000,.\BTsImages\1600 to 1799,BT-1655 TO BT-2000,xls,.\BTsImages\1600 to 1799\BT-1655 TO BT-2000.xls,2001-05-25 08:34:34.000000,False
22973,Astoria Base and Meridian Lines,\\clatsop.co.clatsop.or.us\data\Applications\S...,2019-01-08 14:34:37.000000,Astoria Base and Meridian Lines B0 P08,.\Subdivisions_by_Name_All,Astoria Base and Meridian Lines B0 P08,jgwx,.\Subdivisions_by_Name_All\Astoria Base and Me...,2021-11-09 14:35:34.635747,False


In [33]:
import numpy as np
import datetime
dti = pd.to_datetime(
    ["1/1/2018", np.datetime64("2018-01-01"), datetime.datetime(2018, 1, 1)]
)
dti


DatetimeIndex(['2018-01-01', '2018-01-01', '2018-01-01'], dtype='datetime64[ns]', freq=None)

In [34]:
type(dti)

pandas.core.indexes.datetimes.DatetimeIndex

In [35]:
from datetime import datetime
floatytime = datetime.fromtimestamp(1566924882.689)
type(floatytime)
pd.to_datetime(floatytime)

Timestamp('2019-08-27 09:54:42.689000')